In [1]:
import os
import numpy as np
import scipy.io as sio
import tensorflow as tf

In [ ]:
file_dir = '../../OneDrive - Universidad Católica de Chile/Documents/MRI-Datasets/Anon_Attilio/001/'
acq_file = '2D_NSA1_24S_2MM_IM_0017_ANON/2D_NSA1_24S_2MM.mat'
mat = sio.loadmat(file_dir + acq_file)

In [ ]:
acq = mat['imDataParams'][0,0][0].astype('complex64')
TE = mat['imDataParams'][0,0][1].astype('float32')

In [ ]:
TE.shape

In [ ]:
file_dir = '../../OneDrive - Universidad Católica de Chile/Documents/MRI-Datasets/Anon_Attilio/001/'
out_file = 'results_MP_GC_IM_0017_ANON/2D_NSA1_24S_2MM_MP_GC.mat'
out_mat = sio.loadmat(file_dir + out_file)

In [ ]:
print(out_mat.keys())
out_wf = out_mat['R']
out_r2 = out_mat['R2']
out_fm = out_mat['P']

In [ ]:
out_wf.shape

## Create TFRecord file

In [2]:
def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [3]:
file_dir = '../../OneDrive - Universidad Católica de Chile/Documents/MRI-Datasets/Anon_Attilio/'
recordPath = "tfrecord/"
writer = tf.io.TFRecordWriter(recordPath + 'TFR-Attilio')

keys = [("%03d" % i) for i in list(range(1,10))]
values = [i for i in list(range(10))]
classes = dict(zip(keys, values))

for name, label in classes.items():
    print('Subject No. ' + name)
    subj_path = os.path.join(file_dir, name)
    for file_name in os.listdir(subj_path):
        if '2D_' in file_name:
            print('\t' + 'Processing ' + file_name)
            img_path = os.path.join(subj_path, file_name)
            img_path = os.path.join(img_path, os.listdir(img_path)[0])
            img_mat = sio.loadmat(img_path)
            acq = img_mat['imDataParams'][0,0][0].astype('complex64')
            TE = img_mat['imDataParams'][0,0][1].astype('float32')
            acq_real = np.real(acq)
            acq_imag = np.imag(acq)
            example = tf.train.Example(features=tf.train.Features(feature={
                'acq_real': _float_feature(acq_real.flatten()),
                'acq_imag': _float_feature(acq_imag.flatten()),
                'te': _float_feature(TE.flatten()),
                'height': _int64_feature(acq.shape[0]),
                'width': _int64_feature(acq.shape[1]),
                'num_slices': _int64_feature(acq.shape[2]),
                'num_echoes': _int64_feature(acq.shape[4])}))
            writer.write(example.SerializeToString())
        if 'results_' in file_name:
            print('\t' + 'Processing ' + file_name)
            out_path = os.path.join(subj_path, file_name)
            out_path = os.path.join(out_path, os.listdir(out_path)[0])
            out_mat = sio.loadmat(out_path)
            wf = out_mat['R'].astype('complex64')
            wf_real = np.real(wf)
            wf_imag = np.imag(wf)
            r2 = out_mat['R2'].astype('float32')
            fm = out_mat['P'].astype('float32')
            example = tf.train.Example(features=tf.train.Features(feature={
                "wf_real": _float_feature(wf_real.flatten()),
                "wf_imag": _float_feature(wf_imag.flatten()),
                "r2": _float_feature(r2.flatten()),
                "fm": _float_feature(fm.flatten()),
                'height': _int64_feature(wf.shape[0]),
                'width': _int64_feature(wf.shape[1]),
                'num_slices': _int64_feature(wf.shape[2]),
                'num_specie': _int64_feature(wf.shape[3])}))
            writer.write(example.SerializeToString())
writer.close()

Subject No. 001
	Processing 2D_NSA1_24S_2MM_IM_0017_ANON
	Processing results_MP_GC_IM_0017_ANON
Subject No. 002
	Processing 2D_NSA1_24S_2MM_IM_0017_ANON
	Processing results_MP_GC_IM_0017_ANON
Subject No. 003
	Processing 2D_NSA1_24S_2MM_IM_0013_ANON
	Processing results_MP_GC_IM_0013_ANON
Subject No. 004
	Processing 2D_NSA1_24S_2MM_IM_0010_ANON
	Processing results_MP_GC_IM_0010_ANON
Subject No. 005
	Processing 2D_NSA1_24S_2MM_IM_0010_ANON
	Processing results_MP_GC_IM_0010_ANON
Subject No. 006
	Processing 2D_NSA1_24S_2MM_IM_0007_ANON
	Processing results_MP_GC_IM_0007_ANON
Subject No. 007
	Processing 2D_NSA1_24S_2MM_IM_0010_ANON
	Processing results_MP_GC_IM_0010_ANON
Subject No. 008
	Processing 2D_NSA1_24S_2MM_IM_0007_ANON
	Processing results_MP_GC_IM_0007_ANON
Subject No. 009
	Processing 2D_NSA1_24S_2MM_IM_0010_ANON
	Processing results_MP_GC_IM_0010_ANON


## Load images from TFRecord